In [1]:
from transformers import AutoProcessor, AutoModelForVision2Seq


print("Loading model...")# Load model directly

processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf",device_map="cuda:0")
model = AutoModelForVision2Seq.from_pretrained("llava-hf/llava-1.5-7b-hf",device_map="cuda:0", dtype="bfloat16")





/home/zhaoxu.li/.conda/envs/EHA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/projects/_ssd/ZhaoxuCode/Efficient-HA/transformers-4.57.3/src/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading model...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/projects/_ssd/ZhaoxuCode/Efficient-HA/transformers-4.57.3/src/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.30s/it]


In [2]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "/projects/_hdd/Datazx/coco_val_images/val2014/COCO_val2014_000000581929.jpg"},
            {"type": "text", "text": "Describe the image in detail."}
        ]
    },
]
inputs = processor.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

In [ ]:
print(inputs['input_ids'].shape)

torch.Size([1, 595])


In [4]:

lm_head = model.lm_head
norm = model.language_model.norm
model.set_attn_implementation('eager')
position = {
                            "image_start": inputs['input_ids'].tolist()[0].index(32000), 
                            "image_end": inputs['input_ids'].tolist()[0].index(32000)+24*24, 
                        }
outputs = model.generate(**inputs, max_new_tokens=512,  do_sample=False, use_ours=True,alpha=0.6, threshold_top_p=0.9, threshold_top_k=20,
                                        early_exit_layers=[i for i in range(18,26)], lm_head=lm_head,
                norm=norm, position=position)

using our method


In [5]:
generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, outputs)
    ]
output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]

In [6]:
print(output_text
      )

The image depicts a grassy field where two horses are grazing on grass. One horse is positioned towards the left side of the field, while the other horse is located more centrally within the field. Both horses are enjoying their time grazing on the grassy field.
